In [19]:
import cv2
import numpy as np
import pywt


# Arnold Transform function
def arnold_transform(image, iterations):
    rows, cols = image.shape
    transformed_image = np.copy(image)
    for _ in range(iterations):
        new_image = np.zeros_like(transformed_image)
        for y in range(rows):
            for x in range(cols):
                new_x = (2 * x + y) % cols
                new_y = (x + y) % rows
                new_image[new_y, new_x] = transformed_image[y, x]
        transformed_image = new_image
    return transformed_image


# Partition image into 16x16 blocks
def partition_image(image, block_size=16):
    h, w = image.shape
    blocks = [image[i:i+block_size, j:j+block_size] for i in range(0, h, block_size) for j in range(0, w, block_size)]
    return blocks

# Calculate HVS for a block
def calculate_hvs(block):
    return np.std(block)

# Embed watermark bit into the SVD of the block
def embed_watermark(U, s, V, watermark_bit, T):
    s[0] += T * watermark_bit
    return U, s, V

def full_watermark_embedding(original_image_path, watermark_path, save_path, T=0.1, iterations=5):
    # Read the original image and convert it to grayscale
    original_image = cv2.imread(original_image_path)
    grayscale_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

    # Read the watermark image
    watermark_image = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)
    # Resize the watermark image to 64x112 if not already
    if watermark_image.shape[:2] != (64, 112):
        watermark_image = cv2.resize(watermark_image, (64, 112))

    _, binary_watermark = cv2.threshold(watermark_image, 127, 255, cv2.THRESH_BINARY)
    arnold_watermark = arnold_transform(binary_watermark, iterations)

    blocks = partition_image(grayscale_image, block_size=16)
    hvs_values = np.array([calculate_hvs(block) for block in blocks])
    sorted_indices = np.argsort(hvs_values)[-len(blocks):]  # Select all blocks

    watermarked_image = np.copy(grayscale_image)
    arnold_watermark_flat = arnold_watermark.flatten()

    # Embed watermark in selected blocks based on HVS
    for i, index in enumerate(sorted_indices[:len(arnold_watermark_flat)]):
        watermark_index = i % len(arnold_watermark_flat)
        block = blocks[index]
        watermark_bit = arnold_watermark_flat[watermark_index]
        coeffs = pywt.dwt2(block, 'haar')
        LL, (LH, HL, HH) = coeffs
        U, s, V = np.linalg.svd(LL, full_matrices=False)
        U, s, V = embed_watermark(U, s, V, watermark_bit, T)
        LL_mod = np.dot(U, np.dot(np.diag(s), V))
        coeffs_mod = (LL_mod, (LH, HL, HH))
        block_reconstructed = pywt.idwt2(coeffs_mod, 'haar')
        row = (index // (grayscale_image.shape[1] // 16)) * 16
        col = (index % (grayscale_image.shape[1] // 16)) * 16
        watermarked_image[row:row+16, col:col+16] = block_reconstructed

    # Save the watermarked image
    cv2.imwrite(save_path, watermarked_image)

# Run the watermark embedding

In [20]:
# Define file paths for the original image, watermark, and the output
original_image_path = '/Users/kabir/Downloads/Watermark/images/1003.png'
watermark_path = '/Users/kabir/Downloads/Watermark/images/watermark_64x112_image.png'
save_path = '/Users/kabir/Downloads/Watermark/images/watermarked_image.png'

In [21]:
full_watermark_embedding(original_image_path, watermark_path, save_path)

ValueError: could not broadcast input array from shape (16,14) into shape (16,16)

In [ ]:

watermarked_image_path = '/Users/kabir/Downloads/Watermark/images/watermarked_image.png'
original_s_values_path = 'C:/Users/ptcfi/OneDrive/Desktop/Spring 2024/CS700B- Project/AIGI/original_s_values.npy'
extracted_watermark_path = '/Users/kabir/Downloads/Watermark/images/ext_watermarked_image.png'

# Arnold Transform Inverse function
def inverse_arnold_transform(image, iterations):
    rows, cols = image.shape
    transformed_image = np.copy(image)
    for _ in range(iterations):
        new_image = np.zeros_like(transformed_image)
        for y in range(rows):
            for x in range(cols):
                new_x = (x - y) % cols
                new_y = (-2 * x + y) % rows
                new_image[new_y, new_x] = transformed_image[y, x]
        transformed_image = new_image
    return transformed_image

# Function to extract watermark
def extract_watermark(watermarked_image_path, original_s_values_path, extracted_watermark_path, T=0.1, iterations=5):
    # Load the watermarked image and original singular values
    watermarked_image = cv2.imread(watermarked_image_path, cv2.IMREAD_GRAYSCALE)
    original_s_values = np.load(original_s_values_path, allow_pickle=True)

    # Partition image into 16x16 blocks
    blocks = partition_image(watermarked_image, block_size=16)
    hvs_values = np.array([calculate_hvs(block) for block in blocks])
    sorted_indices = np.argsort(hvs_values)[-len(blocks):]  # Select all blocks based on HVS

    watermark_bits = []

    # Extract watermark from selected blocks
    for i, index in enumerate(sorted_indices[:len(original_s_values)]):
        block = blocks[index]
        coeffs = pywt.dwt2(block, 'haar')
        LL, (LH, HL, HH) = coeffs
        U, s, V = np.linalg.svd(LL, full_matrices=False)
        original_s = original_s_values[i]

        # Determine the watermark bit based on the difference in the first singular value
        watermark_bit = 1 if s[0] > original_s[0] + T / 2 else 0
        watermark_bits.append(watermark_bit)

In [ ]:
# Arnold Transform function
def arnold_transform(image, iterations):
    rows, cols = image.shape
    transformed_image = np.copy(image)
    for _ in range(iterations):
        new_image = np.zeros_like(transformed_image)
        for y in range(rows):
            for x in range(cols):
                new_x = (2 * x + y) % cols
                new_y = (x + y) % rows
                new_image[new_y, new_x] = transformed_image[y, x]
        transformed_image = new_image
    return transformed_image

arnold_transform('')